In [1]:
import requests
import json
import pandas as pd
from bqplot import pyplot as plt
from bqplot import DateScale, LinearScale, Lines, Axis, Figure, OHLC

In [33]:
ticker = 'SPCE'
interval = 'day'
res = requests.get('http://pipkoff-api:80/api/v0/market/candles', params={'ticker': ticker, 'interval': interval})
cont = json.loads(res.content)
candles = cont['data']['candles']
print('len candles: ', len(candles))
dfc = pd.DataFrame(candles)
dfc["time"] = pd.to_datetime(dfc["time"])
dfc = dfc.set_index("time")
dfc.tail()

len candles:  62


,l,o,c,v,h,interval
time,,,,,,
2021-08-09 04:00:00+00:00,32.82,33.21,35.21,5857499,35.54,day
2021-08-10 04:00:00+00:00,31.00,35.25,31.33,7698722,35.47,day
2021-08-11 04:00:00+00:00,26.49,31.01,27.36,10526196,31.50,day
2021-08-12 04:00:00+00:00,25.51,27.30,25.94,7935037,28.50,day
2021-08-13 04:00:00+00:00,25.37,25.68,25.37,8802600,26.29,day


In [30]:
x_date = DateScale()
y_linear = LinearScale()

ohlc = OHLC(x=dfc.index, y=dfc[["o","h","l","c"]],
            scales={'x':x_date, 'y':y_linear},
            marker="candle",
            stroke="#777", stroke_width=1.0,
            colors=["lime", "tomato"],
            )

ax_x = Axis(scale=x_date, label="Date",
            label_offset="35px", grid_color="gray",
            )
ax_y = Axis(scale=y_linear, label="Price",
            orientation="vertical", label_offset="35px",
            grid_color="gray",
            tick_format="0.1f")

fig = Figure(marks=[ohlc],
             axes=[ax_x, ax_y],
             title="CandleStick Chart",
             fig_margin= dict(top=60, bottom=40, left=50, right=20),
             background_style = {"fill":"#ccc"}
      )

fig.layout.height="600px"

fig


Figure(axes=[Axis(grid_color='gray', label='Date', label_offset='35px', scale=DateScale()), Axis(grid_color='g…